In [1]:
import sys, os


import sklearn
import mne
import wandb
sys.path.insert(1, os.path.realpath(os.path.pardir))



from nilearn import image
from nilearn import plotting

import numpy as np
import matplotlib.pyplot as plt

import mne
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation



## torch libraries 

import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset
from pytorch_model_summary import summary
import numpy as np

# **Change on 28 November**
# - Remove bias in all convolution 
# - Use GELU instead ReLu
# - Change batch on instance  normalization 
#     - It compute statisctics croos all time points for some channels. -> channels have same distribution.
#     - On inference we apply statisctics from training.
#     - Might be issue when we inferecen for all timeserais lenght.- Add more filters per channel in SepConv 
# - dilation the same.

class SepConv1D(nn.Module):
    def __init__(self, nin, kernels_per_layer, nout, kernel_size=3, dilation=1):
        super(SepConv1D, self).__init__()
        self.depthwise = nn.Conv1d(nin, nin * kernels_per_layer, 
                                   kernel_size=kernel_size, 
                                   padding='same', 
                                   groups=nin, 
                                   dilation=dilation, 
                                   bias = False)
        self.pointwise = nn.Conv1d(nin * kernels_per_layer, nout, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out
    
    
class Conv_block(nn.Module):
    """
    Input is [batch, emb, time]
    simple conv block from wav2vec 2.0 
        - conv
        - layer norm by embedding axis
        - activation
    """
    def __init__(self, in_channels, out_channels, kernel_size=9, stride=1, dilation=1):
        super(Conv_block, self).__init__()
        
        # use it instead stride. 
        self.downsample = nn.AveragePool1d(kernel_size=stride, stride=stride)
        
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, bias=False)
        self.norm = nn.LayerNorm1d(out_channels) 
        self.activation = nn.GELU()

        
        
    def forward(self, x):
        
        x = self.downsample(x)
        x = self.conv1d(x)
        x = self.norm(x)
        x = self.activation(x)
        
        return x
        
    
class UpsampleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, scale=2):
        super(UpsampleConvBlock, self).__init__()
        
        self.upsample = nn.Upsample(scale_factor=scale, mode='linear')
        self.conv_block = Block1D(in_channels, out_channels, kernel_size)
            
    def forward(self, x_small):
        
        x_upsample = self.upsample(x_small)
        x = self.conv_block(x_upsample)
        
        return x
        
        
        
class AutoEncoder1D(nn.Module):
    """
    This is implementaiotn of AutoEncoder1D model for time serias regression
    Try to adopt fully convolutional network for fMRI decoding 
    1. Denoise stage - Use res blocks for it 
    2. Encode information using 1D convolution with big stride(now use max pool)
    3. Decode information using cat pass from previous levels.
    4. Map to only 21 ROI
    
    hp_autoencoder = dict(  n_res_block = 0, 
                            channels = [32, 32, 64, 64], 
                            kernel_sizes = [15, 11, 5, 5],
                            strides= [8, 8, 4],
                            n_channels_out = 21,
                            n_channels_inp = 900)


    model = AutoEncoder1D(**hp_autoencoder)
    print(summary(model, torch.zeros(4, 30, 30 , 1024), show_input=False))

    
    Time lenght of input and output the same. 
    """
    def __init__(self,n_electrodes=30,
                 n_freqs = 16,
                 n_channels_out=21,
                 n_res_block=1,
                 channels = [8, 16, 32], 
                 kernel_sizes=[3, 3, 3],
                 strides=[4, 4, 4]):
        
        super(AutoEncoder1D, self).__init__()
        
        self.n_res_block = n_res_block
        self.model_depth = len(channels)-1

        self.n_electrodes = n_electrodes
        self.n_freqs = n_freqs
        
        ## factorized features reducing
        # electrodes spatial reducing
        self.spatial_reduce_2d = nn.Conv2d(self.n_electrodes, 16, kernel_size=1)
        
        # freqs-electrodes  reducing to channels[0].
        self.spatial_reduce = nn.Conv1d(16*self.n_freqs, channels[0], kernel_size=1)
        
        
        # create downsample blcoks in Sequentional manner.
        self.downsample_blocks = nn.ModuleList([Block1D(channels[i], 
                                                        channels[i+1], 
                                                        kernel_sizes[i],
                                                        stride=strides[i], 
                                                        dilation=2) for i in range(self.model_depth)])
        
        
        # mapping in latent space
        self.mapping = Block1D(channels[-1], channels[-1],
                               kernel_sizes[-1], 
                               stride=1,
                               dilation=1)
        
        self.conv1x1_one = nn.Conv1d(channels[-1], n_channels_out, kernel_size=1, padding='same')

        
        # upscale one time to. Inverse Interpolation. 
        scale_one = 2**int(np.sum(np.log2(strides)))
        self.upsample_one_time = nn.Upsample(scale_factor=scale_one, mode='linear')


    def forward(self, x):
        """
        1. Denoise 
        2. Encode  
        3. Mapping 
        4. Decode 
        """
        batch, elec, n_freq, time = x.shape
        
        # x = x.reshape(batch, self.n_electrodes, -1, time)
        x = self.spatial_reduce_2d(x)
        
        x = x.reshape(batch, -1, time)
        x = self.spatial_reduce(x)
        
        # encode information
        for i in range(self.model_depth):
            x = self.downsample_blocks[i](x)            
        
        # make mapping 
        x = self.mapping(x)
        x = self.conv1x1_one(x)
        
        # inverse interpolation
        x = self.upsample_one_time(x)
        
        return x


In [60]:
class Conv_block(nn.Module):
    """
    Input is [batch, emb, time]
    simple conv block from wav2vec 2.0 
        - conv
        - layer norm by embedding axis
        - activation
    """
    def __init__(self, in_channels, out_channels, kernel_size=9, stride=1, dilation=1):
        super(Conv_block, self).__init__()
        
        # use it instead stride. 
        self.downsample = nn.AvgPool1d(kernel_size=stride, stride=stride)
        
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, bias=False, 
                                padding='same')
        
        self.norm = nn.LayerNorm(out_channels) 
        self.activation = nn.GELU()

        
        
    def forward(self, x):
        
        x = self.downsample(x)
        x = self.conv1d(x)
        
        # norm by last axis.
        x = torch.transpose(x, -2, -1) 
        x = self.norm(x)
        x = torch.transpose(x, -2, -1) 
        
        
        x = self.activation(x)
        
        return x
    
class Wav2vec_Conv(nn.Module):
    """


    """
    def __init__(self,
                 n_inp_features=30,
                 channels = [8, 16, 32], 
                 kernel_sizes=[3, 3, 3],
                 strides=[4, 4, 4], 
                 dilations = [1, 1, 1]):
        
        super(Conv_Wav2vec, self).__init__()
        
        # freqs-electrodes  reducing to channels[0].
        channels = [n_inp_features] + channels
        
        self.model_depth = len(channels)-1
        # create downsample blcoks in Sequentional manner.
        self.downsample_blocks = nn.ModuleList([Conv_block(channels[i], 
                                                        channels[i+1], 
                                                        kernel_sizes[i],
                                                        stride=strides[i], 
                                                        dilation=dilations[i]) for i in range(self.model_depth)])
        
    def forward(self, x):
        """
        1. Encode  
        """
        batch, n_freq, time = x.shape
    
        # encode information
        for block  in self.downsample_blocks:
            x = block(x)
        return x


In [ ]:
class Wav2Vec_aggregate(nn.Module):
    """
    Inpy should be. 
    batch, n_ch, n_freq, time = x.shape
    
    model_embedding - should be work with [batch, n_freqs, time]
    
    Return [batch, n_ch, emb, time//stride] 
    """
    def __init__(self,
                 model_embedding,
                 ):
        
        super(Wav2Vec_aggregate, self).__init__()
        
        # freqs-electrodes  reducing to channels[0].
        channels = [n_inp_features] + channels
        
        # create downsample blcoks in Sequentional manner.
        self.embedding = model_embedding
        
    def forward(self, x):
        """
        1. Encode  
        """
        batch, n_ch, n_freq, time = x.shape
        
        emb_list  = []
        for ch in range(n_ch):
            x_ch = self.embedding(x[:, ch])
            
            emb_list.append(x_emb)
        
        return x

In [61]:
encoder.model_depth

3

In [71]:
encoder = Conv_Wav2vec(n_inp_features=16,
             channels = [64, 64, 64, 64, 64], 
                 kernel_sizes=[9, 5, 5, 3, 3],
                 strides=[4, 4, 4, 2, 2], 
                 dilations = [1, 1, 1, 1, 1])

In [72]:
eeg = torch.ones([2, 64, 16, 4096])

# block = Conv_block(in_channels=16, out_channels=64, kernel_size=9, stride=4)

eeg_out_2 = encoder(eeg[:, 0])
print(eeg_out_2.shape)




bef torch.Size([2, 16, 4096])
after torch.Size([2, 64, 1024])
bef torch.Size([2, 64, 1024])
after torch.Size([2, 64, 256])
bef torch.Size([2, 64, 256])
after torch.Size([2, 64, 64])
bef torch.Size([2, 64, 64])
after torch.Size([2, 64, 32])
bef torch.Size([2, 64, 32])
after torch.Size([2, 64, 16])
torch.Size([2, 64, 16])


In [73]:
print(summary(encoder, eeg[:, 0], show_input=False))


bef torch.Size([2, 16, 4096])
after torch.Size([2, 64, 1024])
bef torch.Size([2, 64, 1024])
after torch.Size([2, 64, 256])
bef torch.Size([2, 64, 256])
after torch.Size([2, 64, 64])
bef torch.Size([2, 64, 64])
after torch.Size([2, 64, 32])
bef torch.Size([2, 64, 32])
after torch.Size([2, 64, 16])
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
      Conv_block-1       [2, 64, 1024]           9,344           9,344
      Conv_block-2        [2, 64, 256]          20,608          20,608
      Conv_block-3         [2, 64, 64]          20,608          20,608
      Conv_block-4         [2, 64, 32]          12,416          12,416
      Conv_block-5         [2, 64, 16]          12,416          12,416
Total params: 75,392
Trainable params: 75,392
Non-trainable params: 0
-----------------------------------------------------------------------


In [ ]:
x = torch.ones([2, 64, 128, 32])
batch, ch, emb, time = x.shape



torch.Size([2, 64, 4096])

torch.Size([2, 128, 32])

In [ ]:
class ViT_custom(nn.Module):
    def __init__(self,
                 sequence_length=16, 
                 embed_dim=512,
                 num_heads=4,
                 mlp_ratio=2,
                 attn_dropout=0.1,
                 num_layers=1,                
                 mlp_dropout=0.1,                 
                 n_classes=1, 
                ):
        super().__init__()
        

        self.class_embed = nn.Parameter(torch.zeros(1, 1, embed_dim), requires_grad=True)
        self.pe = nn.Parameter(torch.zeros(1, sequence_length + 1, embed_dim), requires_grad=True)
        
        transformer_layer = nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                            nhead=num_heads, 
                                                            dim_feedforward=embed_dim*mlp_ratio, 
                                                            dropout=attn_dropout, 
                                                            activation='relu', 
                                                            batch_first=True)
        
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)


        self.norm = nn.LayerNorm(embed_dim)
        self.mlp_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim*mlp_ratio),
            nn.Dropout(p=mlp_dropout), 
            nn.GELU(),
            nn.Linear(embed_dim*mlp_ratio, 1), )
            
        
    def forward(self, x):
        """
        x.shape - [batch, time , features]
        
        """
        batch, time, features = x.size()
        
        # repeating like batch size and add to batch.
        class_token = self.class_embed.expand(batch, -1, -1)
        x = torch.cat((class_token, x), dim=1)
        
        # add positional embeddings
        x += self.pe
        x = self.transformer(x)
        
        # take first vector -> normalize -> mlp into 
        x_cls = x[:, 0]
        x_cls = self.mlp_head(self.norm(x_cls))
        return x_cls
    
    
transformer_settings = dict(sequence_length=16, 
                             embed_dim=256,
                             num_heads=2,
                             num_layers=2, 
                             mlp_ratio=2,
                             attn_dropout=0.3,                
                             mlp_dropout=0.5,                 
                             n_classes=1)
vit = ViT_custom(**transformer_settings)